In [ ]:
import glob
import os
import sys
from random import sample

import cv2
import numpy as np
import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim



In [ ]:
# helper functions

def images_to_probs(net, images):
    '''
    Generates predictions and corresponding probabilities from a trained
    network and a list of images
    '''
    output = net(images)
    # convert output probabilities to predicted class
    _, preds_tensor = torch.max(output, 1)
    preds = np.squeeze(preds_tensor.numpy())
    return preds, [F.softmax(el, dim=0)[i].item() for i, el in zip(preds, output)]


def plot_classes_preds(net, images, labels):
    '''
    Generates matplotlib Figure using a trained network, along with images
    and labels from a batch, that shows the network's top prediction along
    with its probability, alongside the actual label, coloring this
    information based on whether the prediction was correct or not.
    Uses the "images_to_probs" function.
    '''
    preds, probs = images_to_probs(net, images)
    # plot the images in the batch, along with predicted and true labels
    fig = plt.figure(figsize=(12, 48))
    for idx in np.arange(4):
        ax = fig.add_subplot(1, 4, idx+1, xticks=[], yticks=[])
        matplotlib_imshow(images[idx], one_channel=True)
        ax.set_title("{0}, {1:.1f}%\n(label: {2})".format(
            classes[preds[idx]],
            probs[idx] * 100.0,
            classes[labels[idx]]),
                    color=("green" if preds[idx]==labels[idx].item() else "red"))
    return fig

# helper function to show an image
# (used in the `plot_classes_preds` function below)
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))
        

In [ ]:
data_transform = transforms.Compose([
        transforms.RandomResizedCrop(400),
        transforms.ToTensor()
    ])

classes = ('Other', 'Plans')


In [ ]:
dataset = datasets.ImageFolder(root='../images/root_data',
                                transform=data_transform,
                               )

train_dataset, test_dataset = torch.utils.data.random_split(dataset,
                                                            [1400, 721])


In [ ]:
trainloader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=50,
                                               shuffle=True)

testloader = torch.utils.data.DataLoader(test_dataset,
                                             batch_size=50,
                                             shuffle=True)

In [ ]:
# Device configuration
device = torch.device('cuda:0' 
                      if torch.cuda.is_available()
                      else 'cpu'
                     )
# Hyper parameters
num_epochs = 20
num_classes = 2
batch_size = 50
learning_rate = 0.001
# Specify desired image format
desired = 400
# Specify desired size of test set
split = 20
read_model = True
dim=3

In [ ]:
class ConvNet3d(nn.Module):
    def __init__(self, num_classes=2):
        super(ConvNet3d, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 8, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(50*50*8, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [ ]:
model = ConvNet3d(num_classes).to(device)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)    

In [ ]:
writer = SummaryWriter('logs')

In [ ]:
# get some random training images
dataiter = iter(dataset_loader)
images, labels = dataiter.next()
img_grid = torchvision.utils.make_grid(images)
writer.add_image('plan_images', img_grid)

In [ ]:
# Add graph
writer.add_graph(model, images)
writer.close()

## Add a projector to tensorboard

In [ ]:
running_loss = 0.0
# Train the model
total_step = len(trainloader)

for epoch in range(1):
    for i, (images, labels) in enumerate(trainloader):
        images = images.to(device)
        labels = labels.to(device)
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
               .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
        
        running_loss += loss.item()
        
        # SAVE LOGS
        # ...log the running loss
        writer.add_scalar('training loss',
                        running_loss,
                        epoch * len(trainloader) + i)

        # ...log a Matplotlib Figure showing the model's predictions on a
        # random mini-batch
        writer.add_figure('predictions vs. actuals',
                          plot_classes_preds(model, images, labels),
                          global_step = epoch * len(trainloader) + i)
        running_loss = 0.0